## FURTHER TEXT ANALYSIS OF AMAZON CUSTOMER REVIEW THROUGH TOPIC MODELLING

### Objectives of this test

- Implement **Latent Dirichlet Allocation (LDA) from Gensim package** along with the Mallet’s implementation (via Gensim).

- Implement **Mallet that optimizes LDA**. Mallet is known to run faster and gives better topic segregation.

- Also **extract the volume and percentage contribution of each topic** to get an idea of how important a topic is.

## DATA PREPARATION

### I. Installing libraries and dependencies

**Pre-requisites**: Downloading NLTK Dutch stopwords, data handling tools, model preprocessing & plotting tools, and SpaCy model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Python data analysis tools and python module for printing
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [ ]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [ ]:
# SpaCy for lemmatization
import spacy

In [ ]:
# Plotting tools
!pip install pyLDAvis

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import matplotlib.pyplot as plt

In [ ]:
# Ignoring warnings
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

**PREPARING STOP WORDS** 

In [ ]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words ('english')

### II. Importing data set, exploratory data analysis, and preprocessing

The dataset that I am using is the **amazon personal care appliances reviews** which is a subset of the large Amazon Product review. The dataset is already stored in the **TensorFlow database** and can be loaded directly using the ‘tfds‘ API from Tensorflow.

The dataset consists of reviews of **Amazon Personal_Care_Appliances_v1_00 products in US marketplace**.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
ds = tfds.load('amazon_us_reviews/Personal_Care_Appliances_v1_00', split='train', shuffle_files=True)
assert isinstance(ds, tf.data.Dataset)
print(ds)

In [ ]:
# Converting dataset into a pandas data frame using ‘tfds.as_dataframe‘ API.
df = tfds.as_dataframe(ds)

In [ ]:
# Getting a glimpse of the data frame
df.head()

In [ ]:
# Gettingt the number of rows and columns
df.shape

In [ ]:
# Getting useful information about the data frame
# Note that the column containing the labels (i.e. 'star_rating') is integer in type
df.info()

In [ ]:
# Getting statistical information aboutt the data frame
# Based on the count, mean, and max values, the data frame contains data that is normalised more or less
df.describe()

**IMPUTING 'SENTIMENT' AND 'SHORT_REVIEW' COLUMNS IN THE DATA FRAME**

The **rating provided by the customer is on a scale of 1-5** ( 5 being the highest). As I am going to implement a binary classification model, I shall be converting these ratings into 2 categories, i.e 1 and 0. **Ratings above and equal to 3 will be labeled as Positive (1)** and **below 3 will be negative (0**). The following code will help us implement these steps.

In [ ]:
df['Sentiment'] = df['data/star_rating'].apply(lambda score: 'positive' if score >= 3 else 'negative')
df['Sentiment'] = df['Sentiment'].map({'positive':1, 'negative':0})

In [ ]:
df['short_review'] =df['data/review_body'].str.decode("utf-8")

In [ ]:
df = df[['short_review', 'Sentiment']]

In [ ]:
# Getting a glimpse of the new data frame
df.head()

In [ ]:
df.info()

**DISPLAYING NULL VALUES IF ANY IN ANY FEATURE**

In [ ]:
print(df.isna().sum()) 

###III. Data cleaning: Tokenization of text, removing stopwords, making bigrams & trigtrams, and lemmatizing

**TOKENIZATION**

In [ ]:
# Convert to list
data = df.short_review.values.tolist()
print(data[:1])

In [ ]:
# Tokenizing each sentence into a list of words, removing punctuations and unnecessary characters altogether.
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

**CREATING BIGRAMS AND TRIGRAMS**

Bigrams are two (2) words frequently occurring together in the document. Trigrams are three (3) words frequently occurring.

In [ ]:
# Building the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
# Seeing trigram example
print(trigram_mod[bigram_mod[data_words]])

**REMOVING STOPWORDS, MAKING BIGRAMS, TRIGRAMS, AND LEMMATIZATION**

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for text in texts:
        doc = nlp(" ".join(text)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])

##MODEL TRAINING AND OPTIMISATION

###III. Initializing SpaCy's English NLP model (large size)

**LOADING THE MODEL**

In [ ]:
!pip3 install -U spacy
!python -m spacy download en_core_web_lg
#nlp = spacy.load('en_core_web_lg')

**CALLING THE FUNCTIONS IN ORDER**

In [ ]:
# Removing Stop Words
data_words_nostops = remove_stopwords(data_words)

# Forming Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Loading the SpaCy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])

# Doing lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#print(data_lemmatized)

In [ ]:
print(data_words_nostops[:1])

In [ ]:
print(data_words_bigrams[:1])

**CREATING THE DICTIONARY AND CORPUS NEEDED FOR TOPIC MODELLING**

In [ ]:
# Creating Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Creating Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# Viewing corpus
#print(corpus)

In [ ]:
# Viewing corpus
print(corpus[:1])

Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of (word_id, word_frequency).

For example, (0, 1) above implies, word id 0 occurs once in the document. Likewise, word id 1 occurs once too, and so on.

This is used as the input by the LDA model.

If you want to see what word a given id corresponds to, pass the id as a key to the dictionary.

In [ ]:
# Passing the id as a key to the dictionary to see what word a given ID corresponds to
id2word[0]

In [ ]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus]

###IV. Defining and training the LDA model

The LDA model shall be built with 20 different topics where each topic is a combination of keywords and each keyword contributes a certain weightage to the topic.

In [ ]:
# Building LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

**VIEWING THE TOPICS IN THE LDA MODEL**

In [ ]:
# Printing keywords for 20 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

**NOTE**:

The weights reflect how important a keyword is to that topic.

Looking at these keywords, you can guess what this topic could be.

**COMPUTE MODEL PERPLEXITY AND COHERENCE SCORE**

**Model perplexity** and **topic coherence** provide a convenient measure to judge how good a given topic model is.

In [ ]:
# Computing Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Computing Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

**PLOT: VISUALISING TOPIC-KEYWORDS DISTRIBUTION**

In [ ]:
# Feeding the model into the pyLDAvis instance
vis_1 = gensimvis.prepare(lda_model, corpus, id2word)
vis_1


**How to make inferences from pyLDAvis' output**:

Each bubble on the left-hand side plot represents a topic. The larger the bubble, the more prevalent is that topic.

A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.

A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.

Alright, if you move the cursor over one of the bubbles, the words and bars on the right-hand side will update. These words are the salient keywords that form the selected topic.

## **Model optimisation: Mallet's version of LDA**

Up next, we will improve upon this model by using Mallet’s version of LDA algorithm and then we will focus on how to arrive at the optimal number of topics given any large corpus of text.

Gensim provides a wrapper to implement Mallet’s LDA from within Gensim itself.

In [ ]:
# Upgrading Gensim
# Updated to enforce Gensim v3.8 in Colab (the last version to support running topic models via Mallet).
# https://github.com/polsci/colab-gensim-mallet
!pip3 install --upgrade gensim==3.8

In [ ]:
# Installing Mallet
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip

###V. Implementing LDA optimisation from within base model

In [ ]:
mallet_path = 'mallet-2.0.8/bin/mallet'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [ ]:
# Showing Topics
pprint(ldamallet.show_topics(formatted=False))

In [ ]:
# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

**NOTE**: A remarkable increase in coherence score

###VI. Fine-tuning and saving the optimised LDA model

In [ ]:
mallet_model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet, gamma_threshold=0.001, iterations=50)

In [ ]:
from gensim.test.utils import datapath

# Save model to disk.

temp_file = datapath('drive/MyDrive/models/mallet')

mallet_model.save(temp_file)

**PLOT: OPTIMISED TOPIC-KEYWORDS DISTRIBUTION**

In [ ]:
# Feeding the model into the pyLDAvis instance

vis_2 = gensimvis.prepare(mallet_model, corpus, id2word)
vis_2

**FINDING THE OPTIMAL NUMBER OF TOPICS OF THE LDA MODEL**

The approach to finding the optimal number of topics is to build many LDA models with different values of number of topics (k) and pick the one that gives the highest coherence value.

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    
    return model_list, coherence_values

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)


In [ ]:
# Showing graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

**<NOTE: The optimal coherence score shown in this plot lies around 14 number of topics.**>

In [ ]:
# Printing the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

If the coherence score seems to keep increasing, it may make better sense to pick the model that gave the highest CV before flattening out.

<**CHOOSING THE MODEL WITH, , () TOPICS**>

In [ ]:
# Select the model and print the topics
# The best LDA model with the optimal coherence score has 14 topics, which is
# index 2 from the model_list list
optimal_model = model_list[2]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

**FINAL MODEL**

In [ ]:
final_model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(optimal_model, gamma_threshold=0.001, iterations=50)

**VISUALISING FINAL MODEL**

In [ ]:
vis_3 = gensimvis.prepare(final_model, corpus, id2word)
vis_3

**FINDING THE DOMINANT TOPIC IN EACH SENTENCE**

One of the practical application of topic modeling is to determine what topic a given document is about.

To find that, we find the topic number that has the **highest percentage contribution in the data set**.

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_words):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data_words)

In [ ]:
# Formatting
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Showing
df_dominant_topic.head(10)

**FINDING THE MOST REPRESENTATIVE DOCUMENT FOR EACH TOPIC**

With understanding the topic, you can find the documents a given topic has contributed to the most and infer the topic by reading that document.

In [ ]:
# Grouping the top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reseting Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Formating
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Showing
sent_topics_sorteddf_mallet.head()

The tabular output above actually has 20 rows, one each for a topic. It has the topic number, the keywords, and the most representative document. The **Perc_Contribution column** is nothing but the percentage contribution of the topic in the given document.

**TOPIC DISTRIBUTION ACROSS DOCUMENTS**

Finally, we want to understand the volume and distribution of topics in order to judge how widely it was discussed.

In [ ]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenatinate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics[:20]

##EVALUATION

###VII. Predicting results

In [ ]:
import joblib
import pandas
import gensim
import numpy as np
import pandas as pd
import re
import gensim.corpora as corpora
from gensim.test.utils import datapath
from gensim.models import LdaModel
from pprint import pprint

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
path = datapath('drive/MyDrive/models/mallet')

model = LdaModel.load(path)

pattern = r'\"([\w]*)\"'
pat = re.compile(pattern)
topics = {}
for topic in model.print_topics():
    new_text = pat.findall(topic[1])
    topics[topic[0]] = new_text

In [ ]:
topics_df = pd.Series(topics, name='keywords')
topics_df

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import matplotlib.pyplot as plt

In [ ]:
corpus = joblib.load('./models/corpus')

common_corpus = corpora.dictionary.Dictionary(corpus)

In [ ]:
# Feeding the model into the pyLDAvis instance
vis_4 = gensimvis.prepare(model, [common_corpus.doc2bow(text) for text in corpus], common_corpus)
vis_4

In [ ]:
text = '''
I've been painfully plucking my chin hairs and lady moustache for years. This thing does wonders!! The blade is close enough to 
get a smooth shave, and it also has a protector so it doesn't cut your skin. Easy to use; MUCH faster than plucking; and less 
painful than waxing and plucking. Would definitely recommend!!
'''

In [ ]:
from preprocess import preproces

In [ ]:
lemmatized_text = preproces([text])
new_corpus = [common_corpus.doc2bow(text) for text in lemmatized_text]

In [ ]:
scores = {}
for score in model[new_corpus[0]]:
    scores[score[0]] = np.round(score[1],3)
score_df =pd.Series(scores, name='score')
topics_df = pd.Series(topics, name='keywords')
df = pd.concat([score_df, topics_df], axis=1)
df.index.name = 'topic'
df_final = df.sort_values(by='score', ascending=False)

In [ ]:
df_final[:10]

In [ ]:
#df_final.to_csv('./data/file.csv')

## CONCLUSION

From all Dutch legal text files that were cleaned, further ML preprocessing and unsupervised topic extraction were carried out via:

- A basic topic model using Gensim’s LDA;

- pyLDAvis for topic visualisation; and

- Mallet’s LDA implementation.